# Set Up

## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam

## Extra

# Data Collection/Preprocessing

## Collecting Data

In [ ]:
data = pd.read_csv("../outputs/training_metadata.csv")

In [ ]:
data.head()

## Analyzing Data

In [ ]:
left_labels = data['left']
right_labels = data['right']
forward_labels = data['forward']
backward_labels = data['backward']
brake_labels = data['brake']

left_counts = left_labels.value_counts()
right_counts = right_labels.value_counts()
forward_counts = forward_labels.value_counts()
backward_counts = backward_labels.value_counts()
brake_counts = brake_labels.value_counts()

df = pd.DataFrame({
    'Left': left_counts,
    'Right': right_counts,
    'Forward': forward_counts,
    'Backward': backward_counts,
    'Brake': brake_counts
}).transpose()

# Plot the distribution of labels
fig, ax = plt.subplots(figsize=(10, 6))

df.plot(kind='bar', stacked=True, ax=ax)
ax.set_title('Label Distribution')
ax.set_xlabel('Label')
ax.set_ylabel('Frequency')
ax.legend(['0', '1'], title='Label')

plt.tight_layout()
plt.show()

## Preprocessing Data

In [ ]:
def load_image(image_path):
    image = Image.open("../"+image_path)
    image = np.array(image)
    return image

In [ ]:
data['path'][0]

In [ ]:
images = [load_image(image_path) for image_path in data['path']]
images = np.array(images)

In [ ]:
images[0].shape

In [ ]:
# mean = np.mean(images, axis=(0, 1, 2))
# std = np.std(images, axis=(0, 1, 2))
# scaled_images = (images - mean) / std

In [ ]:
targets = data[['left', 'right', 'forward', 'brake']].values

## Splitting Data

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    images, targets, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42)

In [ ]:
len(X_train)

In [ ]:
len(X_test)

In [ ]:
sample_image_path = "../outputs/training_data/10.jpeg"
sample_image = Image.open(sample_image_path)
image_width, image_height = sample_image.size
num_channels = len(sample_image.getbands())

In [ ]:
num_channels

# Neural Network

## Model Set Up

In [ ]:
optimizer = Adam(learning_rate=0.001)

In [ ]:
# model = models.Sequential([
#     layers.Conv2D(32, (3, 3), activation='relu', input_shape=(
#         image_height, image_width, num_channels)),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu',
#                   kernel_regularizer=regularizers.l2(0.0001)),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu',
#                   kernel_regularizer=regularizers.l2(0.0001)),
#     layers.Flatten(),
#     layers.Dense(64, activation='relu',
#                  kernel_regularizer=regularizers.l2(0.0001)),
#     layers.Dropout(0.6),
#     layers.Dense(4, activation='sigmoid')
# ])

In [ ]:
model = models.Sequential([
    layers.Conv2D(128, (3, 3), activation='relu', input_shape=(
        image_height, image_width, num_channels)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(40, activation='relu'),
    layers.Dropout(0.6),
    layers.Dense(4, activation='relu')
])

In [ ]:
model.summary()

## Training Model

In [ ]:
model.compile(optimizer=optimizer, loss='mse')

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

## Testing Model

In [ ]:
test_loss = model.evaluate(X_test, y_test)

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(train_loss) + 1)
plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
image = X_test[5]
plt.imshow(image)
plt.axis('off')
plt.show()

In [ ]:
test_image = image.reshape(1, image_height, image_width, num_channels)
predictions = model.predict(test_image)
print(predictions)

## Prediction Distribution

In [ ]:
predictions_list = []
for num in range(250):
    image = X_test[num]
    test_image = image.reshape(1, image_height, image_width, num_channels)
    prediction = model.predict(test_image)
    predictions_list.append(prediction)

In [ ]:
predictions_list

In [ ]:
for num in range(250):
    print(y_test[num])

## Saving Model Weights

In [ ]:
model.save("../models/CNN_steering_model_gs_64.h5")